In [ ]:
import os
import numpy as np
import plotly.io as pio
from matplotlib import pyplot as plt

import prepare_data
import run_detection
import plot_detection

pio.renderers.default = 'vscode'
%matplotlib inline

# Data Inspection

## Prepare Data

Configure file paths

In [ ]:
DATA_DIR = 'assets/'

ALL_HE_DIR = DATA_DIR + 'All_He/'
SELECT_HE_DIR = DATA_DIR + 'Selected_He/'

IMG_EUV_DIR = DATA_DIR + 'Img_EUV/'

SINGLE_MAPS_DIR = DATA_DIR + 'Output/Single_Maps/'
ENSEMBLE_MAPS_DIR = DATA_DIR + 'Output/Ensemble_Maps_v0_3/'
ENSEMBLE_MAP_SAVE_DIR = ENSEMBLE_MAPS_DIR + 'Numpy_Arrays/'

Rename Data
- He I and magnetogram FITS files are renamed automatically

In [ ]:
# Delete all gzipped files after renaming
remove_gzip = True

# Rename all He FITS files to include observation date in title
prepare_data.rename_dir(ALL_HE_DIR, remove_gzip)

Specify Date Range
- WSA chs plots and EUV images must be renamed to match the string format DICT_DATE_STR_FORMAT

In [ ]:
# Option to extract all or only select few He FITS files of interest
# All dates
# date_range = ('2000_01_01__00_00', '2020_01_01__00_00')

# CR 2136
# date_range = ('2013_04_01__00_00', '2013_06_01__00_00')

# CR 2151
# date_range = ('2014_06_01__00_00', '2014_06_30__00_00')

# GMU period early in Solar Cycle 24
date_range = ('2012_04_01__00_00', '2012_09_01__00_00')

# Select few dates for analysis
# date_range = None

Extract Data from File System

In [ ]:
# Extract data arrays from He FITS files
HE_FITS_DICT = prepare_data.extract_he(date_range, ALL_HE_DIR, SELECT_HE_DIR)
HE_DATE_LIST = list(HE_FITS_DICT.keys())
HE_DATE_LIST.sort()

# Extract EUV images from image files
EUV_DICT = prepare_data.extract_comparison_ims(IMG_EUV_DIR)

## Available Data

In [ ]:
print('Available Datetimes for He I Observations:')
prepare_data.display_dates(HE_DATE_LIST)

He I FITS File Content

In [ ]:
date_str = '2015_03_31__18_13'
# date_str = '2013_04_17__18_39'
# date_str = '2014_06_06__17_46'

cmaps = [plt.cm.RdBu, plt.cm.afmhot, plt.cm.gray]

plot_detection.plot_fits_content(HE_FITS_DICT, date_str, cmaps)

# Pre-Process

## Single Observation

In [ ]:
date_str = '2012_05_07__17_59'


raw_eqw = HE_FITS_DICT[date_str][0]
eqw_band_cut, eqw_high_cut, eqw_nan = run_detection.pre_process_eqw_v0_1(
    raw_eqw, peak_count_cutoff_percent=0.1
)

arrays = [raw_eqw, eqw_nan, eqw_high_cut, eqw_band_cut]
titles = ['EQW', 'EQW NaN', 'EQW High Cut', 'EQW Band Cut']

plot_detection.plot_hists(arrays[0:2], titles[0:2], semilogy=True)
plot_detection.plot_hists(arrays[2:4], titles[2:4], semilogy=True)

# Parameters

## Threshold

In [ ]:
date_str = '2012_06_22__18_13'

raw_eqw = HE_FITS_DICT[date_str][0]
eqw = run_detection.pre_process_eqw_v0_1(raw_eqw)[0]

plot_detection.plot_thresholds(eqw, bounds=[75, 85, 100], bounds_as_percent=True)

## Structuring Element Radius

In [ ]:
date_str = '2012_06_22__18_13'

raw_eqw = HE_FITS_DICT[date_str][0]
eqw = run_detection.pre_process_eqw_v0_1(raw_eqw)[0]

percent_of_peak_list = [90, 90, 90]
morph_radius_list = [4,10,14]

ch_mask_list = [
    run_detection.get_ch_mask(eqw, percent_of_peak, morph_radius)
    for percent_of_peak, morph_radius
    in zip(percent_of_peak_list, morph_radius_list)
]
title_list = [f'{percent_of_peak:d}% of Peak | {radius:d}px Radius'
              for percent_of_peak in percent_of_peak_list
              for radius in morph_radius_list]
plot_detection.plot_ims([eqw, [[0,0],[0,0]]], ['',''])
plot_detection.plot_ims(ch_mask_list[:3], title_list[:3])

## Smoothness: Gradient Median

In [ ]:
date_str = '2015_03_31__18_13'
percent_of_peak = 90
morph_radius = 15

raw_eqw = HE_FITS_DICT[date_str][0]
eqw = run_detection.pre_process_eqw_v0_1(raw_eqw)[0]

ch_mask = run_detection.get_ch_mask(eqw, percent_of_peak, morph_radius)

plot_detection.plot_sorted_ch_hists(eqw, ch_mask, apply_gradient=True,
                                    hist_stat='median')

# Ensemble

## Individual

In [ ]:
# date_str = '2012_06_22__18_13'
# date_str = '2014_06_26__14_19'
date_str = '2015_03_31__18_13'
# date_str = '2015_07_16__17_12'

percent_of_peak_list = [80,80,90,100,100]
morph_radius_list = [13,17,15,13,17]

raw_eqw = HE_FITS_DICT[date_str][0]
eqw = run_detection.pre_process_eqw_v0_1(raw_eqw)[0]
ensemble_map, isolated_ch_im_list, confidence_list = run_detection.get_ensemble_v0_3(
    eqw, percent_of_peak_list, morph_radius_list
)
plot_detection.plot_ensemble(
    eqw, ensemble_map, confidence_list, isolated_ch_im_list
)

## Output

Create Ensemble Maps & Save to Files (Expensive computation)

In [ ]:
overwrite = False

# v0.2
# O X X
# X O O
# X O X
percent_of_peak_list = [80,80, 90, 100,100]
morph_radius_list = [15,17, 13, 13,17]

# date_idx = 48 # 32: 05/06-08   # 48: 05/26-28

if not os.path.isdir(ENSEMBLE_MAP_SAVE_DIR):
    os.makedirs(ENSEMBLE_MAP_SAVE_DIR)

# for he_date_str in HE_DATE_LIST[date_idx:date_idx + 3]:
for he_date_str in HE_DATE_LIST:
    
    # Optionally overwrite existing files
    save_file = f'{ENSEMBLE_MAP_SAVE_DIR}{he_date_str}_ensemble_map.npy'
    if os.path.isfile(save_file) and not overwrite:
        print((f'He {he_date_str} ensemble map already exists.'))
        continue
    
    raw_eqw = HE_FITS_DICT[he_date_str][0]
    eqw = run_detection.pre_process_eqw_v0_1(raw_eqw)[0]
    # eqw = normalize(raw_eqw)[0]

    # ensemble_map = run_detection.get_ensemble(
    #     eqw, percent_of_peak_list, morph_radius_list
    # )[0]
    ensemble_map = run_detection.get_smooth_ensemble(
        eqw, percent_of_peak_list, morph_radius_list
    )[0]
    
    save_list = [he_date_str, percent_of_peak_list, morph_radius_list, ensemble_map]
    np.save(save_file, np.array(save_list, dtype=object), allow_pickle=True)
    print(f'{he_date_str} Ensemble Map Saved')

Save Ensemble Maps to Images

In [ ]:
output_dir = ENSEMBLE_MAPS_DIR + 'Preprocess_v0_3/2012_05_27/'
date_idx = 48 # 32: 05/06-08   # 48: 05/26-28


if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

for he_date_str in HE_DATE_LIST[date_idx:date_idx + 3]:
    
    raw_eqw = HE_FITS_DICT[he_date_str][0]
    eqw = run_detection.pre_process_eqw_v0_1(raw_eqw)[0]
    # eqw = normalize(raw_eqw)[0]
    
    save_file = f'{ENSEMBLE_MAP_SAVE_DIR}{he_date_str}_ensemble_map.npy'
    ensemble_map = run_detection.get_saved_ensemble_map(save_file)[-1]

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))
    ax = axes.ravel()

    ax[0].set_title(he_date_str, fontsize=24)
    ax[0].imshow(eqw, cmap=plt.cm.gray)
    
    ax[1].imshow(ensemble_map, cmap=plt.cm.magma)

    plt.savefig(f'{output_dir}{he_date_str}.jpg')
    plt.close(fig)

Write Images to Video

In [ ]:
output_dir = ENSEMBLE_MAPS_DIR + 'Preprocess_v0_3/2012_05_27/'

fps = 2

run_detection.write_ensemble_video(output_dir, fps)